In [1]:
import pandas as pd
import numpy as np
import os

os.chdir("/Users/PraveenGupta/Downloads/Kaggle/numerai_datasets/New Outputs/CV/")
i=0
# MergedDF1=None
MergedDF2=pd.read_csv("/Users/PraveenGupta/Downloads/Kaggle/numerai_datasets/numerai_training_data.csv")
MergedDF2=MergedDF2[['t_id','target']]
for file in os.listdir("/Users/PraveenGupta/Downloads/Kaggle/numerai_datasets/New Outputs/CV/"):
    if file.endswith(".csv"):
        i=i+1
#         currentDF = pd.read_csv("/Users/PraveenGupta/Downloads/Kaggle/numerai_datasets/CV/CV_alg_DTree.csv")
        currentDF = pd.read_csv(file)
        MergedDF2 = pd.merge(MergedDF2,currentDF, on=['t_id'])
#         print MergedDF2
#         if i==0:
#             MergedDF2 = currentDF.copy()
#         else:
#             MergedDF2 = pd.merge(MergedDF2,currentDF, on=['t_id'])
MergedDF2.to_csv("Merged/MERGED.csv",index=False)

In [89]:
currentDF.head()

,t_id,CV_alg_DTree
0,A1,0.534462
1,A2,0.534462
2,A3,0.513412
3,A4,0.408747
4,A5,0.525126


In [4]:
import pandas as pd
import numpy as np
import os

os.chdir("/Users/PraveenGupta/Downloads/Kaggle/numerai_datasets/New Outputs/submission/")

# MergedDF1=None
Merged_Test=pd.read_csv("/Users/PraveenGupta/Downloads/Kaggle/numerai_datasets/New Outputs/numerai_tournament_data.csv")
Merged_Test=Merged_Test[['t_id']]
for file in os.listdir("/Users/PraveenGupta/Downloads/Kaggle/numerai_datasets/New Outputs/submission/"):
    if file.endswith(".csv"):
        currentDF = pd.read_csv(file)
        currentDF=currentDF.rename(columns={'probability': 'CV_'+file[:-4]})
        Merged_Test = pd.merge(Merged_Test,currentDF, on=['t_id'])
Merged_Test.to_csv("Merged/MERGED.csv",index=False)

In [4]:
MergedDF2.head()

,t_id,target,CV_alg_DTree,CV_alg_DTree_Normalize_L1,CV_alg_DTree_Robust,CV_alg_ET,CV_alg_ET_Normalize_L1,CV_alg_ET_Robust,CV_alg_GBM,CV_alg_GNB,...,CV_alg_KNN,CV_alg_Logistic_AdaB,CV_alg_Logistic_Normalize_L1,CV_alg_Logistic_Robust,CV_alg_RandomForest,CV_alg_RandomForest_Normalize_L1,CV_alg_RandomForest_Robust,CV_alg_SGD,CV_alg_svm,CV_xgBoost
0,A1,1,0.534462,0.524238,0.534462,0.529417,0.519693,0.529417,0.528096,0.595730,...,0.7,0.500522,0.526132,0.513016,0.529785,0.524143,0.529785,0.529785,0.529785,0.543261
1,A2,0,0.534462,0.524238,0.534462,0.529704,0.519693,0.529704,0.519974,0.604800,...,0.5,0.500407,0.515874,0.518912,0.532368,0.522714,0.532368,0.532368,0.532368,0.502714
2,A3,0,0.513412,0.500187,0.513412,0.514228,0.494142,0.514228,0.530777,0.538378,...,0.6,0.500322,0.504078,0.512147,0.509858,0.498313,0.509800,0.509858,0.509858,0.485308
3,A4,0,0.408747,0.363395,0.408747,0.446607,0.423077,0.446607,0.400990,0.258875,...,0.4,0.498793,0.424148,0.438537,0.425587,0.392463,0.425587,0.425587,0.425587,0.367861
4,A5,1,0.525126,0.530970,0.525126,0.525140,0.583333,0.525140,0.475217,0.478912,...,0.5,0.499988,0.582984,0.504756,0.510398,0.504306,0.510201,0.510398,0.510398,0.496761


In [10]:
Merged_Test.shape

(13349, 20)

In [5]:
target = 'target'
IDCol = ['t_id']
predictors=[x for x in MergedDF2.columns if x not in [target]+IDCol+["predictions"]]

from sklearn import cross_validation, metrics
from datetime import datetime


def modelfit(alg,dtrain,dtest,predictors,target,IDCol,filename):
    #fit the algorithm on the data
    print datetime.now().time()
    alg.fit(dtrain[predictors],dtrain[target])
    
    #predict training data set
    dtrain["predictions"] = alg.predict_proba(dtrain[predictors])[:,1]
    
    #performs cross validation
    cv_score = cross_validation.cross_val_score(alg,dtrain[predictors],dtrain[target],cv=7,scoring='log_loss')
    
    #Print model report
    print "\nModel Report :"
    print "Train Log Loss : %.4g" % (metrics.log_loss(dtrain[target].values,dtrain["predictions"].values))
    print "CV Score : Mean - %.4g | Std - %.4g | Min - %.4g | Max - %.4g"%(np.mean(cv_score),np.std(cv_score),np.min(cv_score),np.max(cv_score))
    
    #Predict on testing data
    dtest[target]=alg.predict_proba(dtest[predictors])[:,1]
    
    #Export submission file
    IDCol.append(target)
    submission = pd.DataFrame({x : dtest[x] for x in IDCol})
    submission = submission.rename(columns={'target': 'probability'})
    if not os.path.exists("submission"):
        os.makedirs("submission")
    submission.to_csv("submission/"+filename,index=False)
    
    
    cv = pd.DataFrame(dtrain[['t_id','predictions']])
    cv = cv.rename(columns={'predictions': "CV_"+filename[:-4]})
    if not os.path.exists("CV"):
        os.makedirs("CV")
    cv.to_csv("CV/"+"CV_"+filename,index=False)
    
    print datetime.now().time()

In [8]:
# Stacked using Logisitic, however not a good result
from sklearn.linear_model import LogisticRegression
alg3=LogisticRegression()
modelfit(alg3,MergedDF2,Merged_Test,predictors,target,IDCol,'alg_Logistic_Merged.csv')


16:43:07.056000

Model Report :
Train Log Loss : 0.5778
CV Score : Mean - -0.5789 | Std - 0.003483 | Min - -0.5837 | Max - -0.5739
16:43:10.748000


In [11]:
# Stacked using Logisitic, however not a good result
from sklearn.tree import DecisionTreeClassifier
alg2R=DecisionTreeClassifier(max_depth=3,min_samples_split=100, min_samples_leaf=50,max_features='sqrt',random_state=111)
modelfit(alg2R,MergedDF2,Merged_Test,predictors,target,IDCol,'alg_DTree_Merged.csv')

16:46:39.491000

Model Report :
Train Log Loss : 0.676
CV Score : Mean - -0.6758 | Std - 0.004832 | Min - -0.6812 | Max - -0.6682
16:46:40.585000


In [12]:
from sklearn.naive_bayes import GaussianNB
algGNB=GaussianNB()
modelfit(algGNB,MergedDF2,Merged_Test,predictors,target,IDCol,'alg_GNB_Merged.csv')

20:29:16.607000

Model Report :
Train Log Loss : 0.8671
CV Score : Mean - -0.869 | Std - 0.02927 | Min - -0.9208 | Max - -0.8333
20:29:17.434000


In [75]:
# Perform blending
from sklearn.metrics import log_loss
from sklearn.metrics import roc_auc_score
noIter=1
multiplier=1/100.0
maxAns=100
i=0
temp=0

series1=pd.Series(MergedDF2['CV_alg_DTree_Normalize_L1'])
series2=pd.Series(MergedDF2['CV_alg_DTree'])
targetseries=pd.Series(MergedDF2['target'])
for inc in range(0,100,noIter):
    MergedDF2['predBlended']=0
    blendDF1=0
    blendDF1 = series1*(inc*multiplier)+ series2*(100-inc)*multiplier
    temp=log_loss(targetseries.values,blendDF1.values)
    if(temp<maxAns):
        maxAns=temp
        print "Min LogLoss obtained as of now %.7g" %(maxAns)
        Blend1=inc*multiplier
        Blend2=(100-inc)*multiplier
print "Max Blend1 coef = %.7g" %(Blend1) 
print "Max Blend2 coef = %.7g" %(Blend2) 
print "If only series1 = %.7g" %(log_loss(targetseries.values,series1.values))
print "If only series2 = %.7g" %(log_loss(targetseries.values,series2.values))

Min LogLoss obtained as of now 0.6897471
Min LogLoss obtained as of now 0.689698
Min LogLoss obtained as of now 0.6896499
Min LogLoss obtained as of now 0.6896027
Min LogLoss obtained as of now 0.6895566
Min LogLoss obtained as of now 0.6895113
Min LogLoss obtained as of now 0.689467
Min LogLoss obtained as of now 0.6894237
Min LogLoss obtained as of now 0.6893813
Min LogLoss obtained as of now 0.6893399
Min LogLoss obtained as of now 0.6892994
Min LogLoss obtained as of now 0.6892599
Min LogLoss obtained as of now 0.6892213
Min LogLoss obtained as of now 0.6891837
Min LogLoss obtained as of now 0.6891469
Min LogLoss obtained as of now 0.6891111
Min LogLoss obtained as of now 0.6890763
Min LogLoss obtained as of now 0.6890424
Min LogLoss obtained as of now 0.6890093
Min LogLoss obtained as of now 0.6889773
Min LogLoss obtained as of now 0.6889461
Min LogLoss obtained as of now 0.6889159
Min LogLoss obtained as of now 0.6888866
Min LogLoss obtained as of now 0.6888582
Min LogLoss obtain

In [7]:
# Prepare submission based on blending coeff
# CV_alg_Logistic_AdaB - 0.49 and CV_alg_RandomForest_Normalize_L1 = 0.51 LB = 0.69224 --> Blended_Submission_1
# CV_alg_RandomForest_Normalize_L1 - 0.41 and CV_alg_RandomForest = 0.59 LB = 0.69174 --> Blended_Submission_RF_w_wo_N1
#### blended without xgboost following coeff LB = 0.69170, CV = 0.69103####
# # Merged_Test['CV_alg_svm']*0.253+ Merged_Test['CV_alg_RandomForest']*0.382+Merged_Test['CV_alg_Logistic']*0.168+\
# Merged_Test['CV_alg_KNN']*0.07+Merged_Test['CV_alg_GNB']*0.029+Merged_Test['CV_alg_GBM']*0.098

Merged_Test['probability']=Merged_Test['CV_alg_svm']*0.253+ \
                            Merged_Test['CV_alg_RandomForest']*0.382+Merged_Test['CV_alg_Logistic']*0.168+\
                            Merged_Test['CV_alg_KNN']*0.07+Merged_Test['CV_alg_GNB']*0.029+\
                            Merged_Test['CV_alg_GBM']*0.098
    
submission_Blended = pd.DataFrame({"t_id": Merged_Test.t_id, "probability": Merged_Test.probability})
submission_Blended.to_csv("submission/Blended_Submission_except_xgboost.csv", index=False)

In [24]:
log_loss(MergedDF2['target'].values,MergedDF2['CV_alg_DTree_Normalize_L1'].values)

0.68938225913694295

In [9]:
from sklearn.metrics import log_loss
MergedDF2['predBlended'] = MergedDF2['CV_alg_KNN']*0.625+ \
                            MergedDF2['CV_xgBoost']*0.374+MergedDF2['CV_alg_DTree']*0.001
log_loss(MergedDF2['target'].values,MergedDF2['predBlended'].values)

0.65822686907540484

In [120]:
MergedDF2['MaxpredBlended']

0        1.268080e-315
1        1.158321e-315
2         9.992007e-16
3         9.992007e-16
4         3.453878e+01
5         3.453878e+01
6         9.992007e-16
7         3.453878e+01
8         3.453878e+01
9         9.992007e-16
10        9.992007e-16
11        3.453878e+01
12        9.992007e-16
13        3.453878e+01
14        9.992007e-16
15        9.992007e-16
16        9.992007e-16
17        9.992007e-16
18        3.453878e+01
19        3.453878e+01
20        3.453878e+01
21        3.453878e+01
22        9.992007e-16
23        3.453878e+01
24        3.453878e+01
25        3.453878e+01
26        9.992007e-16
27        3.453878e+01
28        3.453878e+01
29        9.992007e-16
             ...      
57741     3.453878e+01
57742     9.992007e-16
57743     3.453878e+01
57744     3.453878e+01
57745     3.453878e+01
57746     3.453878e+01
57747     9.992007e-16
57748     3.453878e+01
57749     3.453878e+01
57750     3.453878e+01
57751     9.992007e-16
57752     9.992007e-16
57753     3

In [114]:
MergedDF2['CV_alg_RandomForest_Robust'].head()

0    0.529785
1    0.532368
2    0.509800
3    0.425587
4    0.510201
Name: CV_alg_RandomForest_Robust, dtype: float64

In [142]:
blendDF.head()

,t_id,0,1,2,3,4,5,6,7,8,...,90,91,92,93,94,95,96,97,98,99
0,A1,0.529785,0.529493,0.529200,0.528907,0.528615,0.528322,0.528030,0.527737,0.040042,...,0.503448,0.503156,0.502863,0.502570,0.502278,0.501985,5.677051e-315,0.501400,0.501107,0.500815
1,A2,0.532368,0.532048,0.531729,0.531409,0.531089,0.530770,0.530450,0.530131,0.040033,...,0.503603,0.503284,0.502964,0.502645,0.502325,0.502005,2.605851e-315,0.501366,0.501047,0.500727
2,A3,0.509800,0.509706,0.509611,0.509516,0.509421,0.509327,0.509232,0.509137,0.040026,...,0.501270,0.501175,0.501080,0.500985,0.500891,0.500796,4.979947e-01,0.500606,0.500512,0.500417
3,A4,0.425587,0.426320,0.427052,0.427784,0.428516,0.429248,0.429980,0.430712,0.039903,...,0.491472,0.492204,0.492937,0.493669,0.494401,0.495133,5.020053e-01,0.496597,0.497329,0.498061
4,A5,0.510201,0.510099,0.509997,0.509895,0.509793,0.509691,0.509589,0.509486,0.039999,...,0.501009,0.500907,0.500805,0.500703,0.500600,0.500498,4.992041e-01,0.500294,0.500192,0.500090


In [144]:
blendDF1.head()

,t_id,0,1,2,3,4,5,6,7,8,...,90,91,92,93,94,95,96,97,98,99
0,A1,0.529785,0.529493,0.529200,0.528907,0.020021,0.025026,0.528030,0.527737,0.040042,...,0.503448,0.503156,0.502863,0.502570,0.502278,0.501985,2.705938e-315,0.501400,2.705938e-315,0.500815
1,A2,0.532368,0.532048,0.531729,0.531409,0.020016,0.025020,0.530450,0.530131,0.040033,...,0.503603,0.503284,0.502964,0.502645,0.502325,0.502005,1.016015e-315,0.501366,1.016015e-315,0.500727
2,A3,0.509800,0.509706,0.509611,0.509516,0.732376,0.025016,0.509232,0.509137,0.751616,...,0.501270,0.501175,0.501080,0.500985,0.500891,0.500796,6.947402e-01,0.500606,6.943606e-01,0.500417
3,A4,0.425587,0.426320,0.427052,0.427784,0.578190,0.024940,0.429980,0.430712,0.603272,...,0.491472,0.492204,0.492937,0.493669,0.494401,0.495133,6.834597e-01,0.496597,6.863639e-01,0.498061
4,A5,0.510201,0.510099,0.509997,0.509895,0.693550,0.024999,0.509589,0.509486,0.714351,...,0.501009,0.500907,0.500805,0.500703,0.500600,0.500498,6.921511e-01,0.500294,6.925594e-01,0.500090


In [176]:
blendDF1.head()


0    0.495517
1    0.495403
2    0.502260
3    0.500683
4    0.501915
dtype: float64

In [ ]:
##############ROUGH###############

In [189]:
blendDF10=MergedDF2[['t_id']]
# idSeries = MergedDF2[['t_id']]
# idSeries=True
for i in range(0,100,noIter):
    blendDF10.loc[blendDF10['t_id']==MergedDF2['t_id'],'prob'] = series1*(i*multiplier)+ series2*(100-i)*multiplier

c:\Anaconda2\lib\site-packages\pandas\core\indexing.py:266: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
c:\Anaconda2\lib\site-packages\pandas\core\indexing.py:426: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [191]:
b=series1*(i*multiplier)+ series2*(100-i)*multiplier

In [298]:
# For all two combinations of columns in a dataframe
from sklearn.metrics import log_loss
from sklearn.metrics import roc_auc_score
noIter=1
multiplier=1/100.0
maxAns=100
i=0
temp=0

ColNum = len(MergedDF2.columns)
arrColNames = MergedDF2.columns.values
for k in range(0,ColNum,1):
    for j in range(k+1,ColNum,1):
        if arrColNames[k].startswith('CV_'):
            if arrColNames[j].startswith('CV_'):
                series1=pd.Series(MergedDF2[arrColNames[k]])
                series2=pd.Series(MergedDF2[arrColNames[j]])
                targetseries=pd.Series(MergedDF2['target'])
                for i in range(0,100,noIter):
                    MergedDF2['predBlended']=0
                    blendDF1 = series1*(i*multiplier)+ series2*(100-i)*multiplier
                    temp=log_loss(targetseries.values,blendDF1.values)
                    if(temp<maxAns):
                        maxAns=temp
#                         print "Min LogLoss obtained as of now %.7g" %(maxAns)
                        Blend1=i*multiplier
                        Blend2=(100-i)*multiplier
#                 print '%s-->%s'%(arrColNames[i],arrColNames[j])
                print "************************************************"
                print "Min LogLoss obtained as of now %.7g" %(maxAns)
                print "Max Blend1 coef = %.7g" %(Blend1) 
                print "Max Blend2 coef = %.7g" %(Blend2) 
                print "If only %s = %.7g" %(arrColNames[k],log_loss(targetseries.values,series1.values))
                print "If only %s = %.7g" %(arrColNames[j],log_loss(targetseries.values,series2.values))
#             print '%d-->%d'%(i,j)

************************************************
Min LogLoss obtained as of now 0.06077041
Max Blend1 coef = 0.23
Max Blend2 coef = 0.77
If only CV_alg_DTree = 0.6897471
If only CV_alg_DTree_Normalize_L1 = 0.6893823
************************************************
Min LogLoss obtained as of now 0.06077041
Max Blend1 coef = 0.23
Max Blend2 coef = 0.77
If only CV_alg_DTree = 0.6897471
If only CV_alg_DTree_Robust = 0.6897471
************************************************
Min LogLoss obtained as of now 0.06077041
Max Blend1 coef = 0.23
Max Blend2 coef = 0.77
If only CV_alg_DTree = 0.6897471
If only CV_alg_ET = 0.6912497
************************************************
Min LogLoss obtained as of now 0.06077041
Max Blend1 coef = 0.23
Max Blend2 coef = 0.77
If only CV_alg_DTree = 0.6897471
If only CV_alg_ET_Normalize_L1 = 0.6919025
************************************************
Min LogLoss obtained as of now 0.06077041
Max Blend1 coef = 0.23
Max Blend2 coef = 0.77
If only CV_alg_DTree = 0

In [197]:

series1=pd.Series(MergedDF2['CV_alg_RandomForest_Normalize_L1'])
series2=pd.Series(MergedDF2['CV_alg_Logistic_AdaB'])

0        0.523933
1        0.525976
2        0.507905
3        0.440229
4        0.508159
5        0.483885
6        0.524124
7        0.526658
8        0.517593
9        0.529045
10       0.504940
11       0.528115
12       0.496928
13       0.504178
14       0.508280
15       0.474476
16       0.491558
17       0.514134
18       0.506671
19       0.442351
20       0.513328
21       0.536248
22       0.495070
23       0.496891
24       0.498666
25       0.484992
26       0.515027
27       0.519795
28       0.492741
29       0.474911
           ...   
57741    0.515281
57742    0.527937
57743    0.516903
57744    0.514509
57745    0.525301
57746    0.473541
57747    0.519531
57748    0.528174
57749    0.483247
57750    0.458239
57751    0.489998
57752    0.517173
57753    0.519594
57754    0.493479
57755    0.499386
57756    0.466165
57757    0.519142
57758    0.509652
57759    0.518326
57760    0.512307
57761    0.480695
57762    0.452941
57763    0.450072
57764    0.477673
57765    0